In [15]:
display(Javascript('''var c = Jupyter.notebook.delete_cell();'''))


<IPython.core.display.Javascript object>

In [16]:
HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>''')


In [17]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def hide_code_fun():
    display(HTML('''<script>$('div.input').hide();</script>'''))

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = Jupyter.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    add_raw_attachment()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        file = open(path_file, "rb")
        image = file.read()
        w=widgets.Image(
            value=image,
            format=ext.replace('.', ''),
            width=300,
            height=400,
        )
        display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def add_raw_attachment():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    attach_list = os.listdir(path_attachments)

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        all_buttons = [update, chosen_attach_wid, confirm]
        display(widgets.HBox(all_buttons))
    else:
        print("ATTENZIONE: non sono presenti file nella cartella allegati")
        update.on_click(on_button_update)
        display(update)

def on_button_delete(_):
    delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = Jupyter.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true
                                IPython.notebook.delete_cell([c-1])'''))
def delete_this_cell():
    display(Javascript('''var c = Jupyter.notebook.delete_cell();'''))

def on_button_raw_attach(_):
    add_code_cell_din('code_button_delete()')
    add_attach_cell_din('add_raw_attachment()')

def on_button_md(_):
    add_code_cell_din('code_button_delete()')
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din('code_button_delete()')
    add_code_cell_din()

def on_button_raw(_):
    add_code_cell_din('code_button_delete()')
    add_raw_cell_din()

def add_raw_cell_din():
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);'''))
    return

def add_attach_cell_din(code = ''):
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false
    IPython.notebook.get_cell(t_index).set_text('');'''))
    return

def add_code_cell_din(code = ''):
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).set_text('');'''))
    return

def add_md_cell_din2(att): # non attivabile al momento
    s_all = '### Hai allegato il file ' + att + ' alla richiesta '
    if '.png' in att or '.gif' in att:
        s_all += '<br><img src=\"' + path + '/' + att + '\" width=500 >'
    display(Javascript('''var c = Jupyter.notebook.insert_cell_below();
        c.set_text(' ''' + s_all + ''' ');
        var t_index = IPython.notebook.get_cells().indexOf(c);
        IPython.notebook.to_markdown(t_index);
        IPython.notebook.get_cell(t_index).render();
        IPython.notebook.execute_cells([t_index]);
        IPython.notebook.get_cell(t_index).set_text('');'''))
    return

def add_md_cell_din():
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);'''))
    return

def code_button_delete():
    button_delete = widgets.Button(description="Elimina", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Programmazione lineare - Metodo delle due fasi #


Si consideri il seguente problema di Programmazione Lineare:<br>max $6x_1 - 5x_2 - 3x_3 - 4x_4$<br>tale che:<br>$4x_1 + 5x_2 - 3x_3 + x4 \le 5$<br>$2x_1 - 5x_2 + x3 - x4 \le -5$<br>$2x_1 - x_2 + 2x_3 - x_4 \le 4$<br>$x_1, x_2, x_3, x_4 \ge 0$

__Richiesta 1 \[10 punti\]__: Impostare il problema ausiliario

In [18]:
loader_main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

__Richiesta 2 \[30 punti\]__: Risolvere il problema ausiliario per ottenere una soluzione ammissibile di base al problema originario

In [19]:
loader_main()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

__Richiesta 3 \[10 punti\]__: Dalla soluzione ottima del problema ausiliario, ricavare un tableau che esprima una soluzione di base ammissibile per il problema originario

In [20]:
loader_main()


__Richiesta 4 \[30 punti\]__: Risolvere il problema originario all'ottimo

In [21]:
loader_main()


__Richiesta 5 \[10 punti\]__: Quanto si sarebbe disposti a pagare per ogni unità di incremento per l'availability nei tre vincoli per piccole variazioni?

In [22]:
loader_main()


__Richiesta 6 \[20 punti\]__: Fino a dove si sarebbe disposti a pagare tali prezzi ombra?

In [23]:
loader_main()


### Documentazione sulla classe Tableau disponibile nella libreria del simplesso interattivo ###
__Nota__: la libreria è già automaticamente caricata in questo foglio.

__Metodi disponibili__:
- t = Tableau(n, m, obj, prob\_type, term\_noto\_obj): definisce un nuovo problema di programmazione lineare avente _n_ variabili e _m_ vincoli; _obj_ sono i coefficienti delle variabili nella funzione obiettivo; _prob\_type_ può essere 'max' o 'min'; _term\_noto\_obj_ è un parametro opzionale casomai nella funzione obiettivo ci sia un valore costante da aggiungere; il tableau è di tipo completo (i.e., mostra nelle colonne tutte le variabili, comprese le slack);
- t.aggiungi\_vincolo(expression, value): aggiunge un vincolo di disuguaglianza '<=' con coefficienti delle variabili i valori contenuti in _expression_ e termine noto _value_ ;
- t.is\_optimal(): restituisce True o False;
- t.is\_feasible(): restituisce True o False;
- t.mostra\_tableau(): stampa il tableau corrente;
- t.crea\_primo\_tableau(): una volta definito il problema e inseriti i vincoli, crea il primo tableau e la prima soluzione di base;
- t.step\_primale(): esegue un passo del simplesso primale sul tableau t;
- t.step\_duale(): esegue un passo del simplesso duale sul tableau t;
- t.pivot\_colonna\_riga(c,r): esegue un'operazione di pivot facendo entrare in base la variabile della colonna _c_ e facendo uscire la variabile della riga _r_ ;
- t.stampa\_soluzione\_base\_corrente();
- t.prossimo\_step(): sulla base della soluzione di base corrente, ritorna informazioni sulla feasibility e l'ottimalità e suggerisce il passo successivo da fare per la risoluzione del problema.


In [24]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun May 3 17:34:36 2020

@author: Alice Raffaele
"""

from __future__ import division
from fractions import Fraction
from numpy import *

class Tableau:

    def __init__(self, n, m, obj, prob_type, term_noto_obj = 0):
        self.n = n
        self.m = m
        self.rows = []
        self.cons = []
        self.basis = []
        self.prob_type = prob_type
        if self.prob_type == 'max':
            self.obj = [Fraction(1)] + [Fraction(x) for x in obj]
            self.term_noto_obj = Fraction(term_noto_obj)
        elif self.prob_type == 'min':
            self.obj = [Fraction(1)] + [-Fraction(x) for x in obj]
            self.term_noto_obj = Fraction(-term_noto_obj)

    def aggiungi_vincolo(self, expression, value):
        self.rows.append([Fraction(0)] + [Fraction(x) for x in expression])
        self.cons.append(Fraction(value))


    def is_optimal(self):
        if max(self.obj[1:-1]) <= 0:
            #print('La soluzione di base associata è ottima.')
            return True
        #print('La soluzione di base associata NON è ottima.')
        return False

    def is_feasible(self):
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        if min(rhs) < 0:
            #print('La soluzione di base associata NON è ammissibile per il primale.')
            return False
        #print('La soluzione di base associata è ammissibile per il primale.')
        return True

    def _colonna_di_max_guadagno(self):
    # restituisce una colonna di pivot utile ad incrementare la funzione obiettivo (come nel simplesso primale)
        low = 0
        idx = 0
        for i in range(1, len(self.obj)-1):
            if self.obj[i] > low:
                low = self.obj[i]
                idx = i
        if idx == 0: return -1
        return idx

    def _riga_max_inammissibile(self):
    # restituisce una riga di pivot utile a ridurre l'inammissibilità (come nel simplesso duale)
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        low = 0
        idx = -1
        for i in range(len(rhs)):
            if rhs[i] < low:
                low = rhs[i]
                idx = i
        return idx

    def _riga_bloccante(self, col):
    # restituisce una riga di pivot per una data colonna di pivot (come nel simplesso primale)
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        lhs = [self.rows[i][col] for i in range(len(self.rows))]
        ratio = []
        for i in range(len(rhs)):
            if lhs[i] <= 0:
                ratio.append(Fraction(99999999 * abs(max(rhs))))
                continue
            ratio.append(Fraction(rhs[i],lhs[i]))
        return argmin(ratio)

    def _colonna_bloccante(self, riga):
    # restituisce una colonna di pivot per una data riga di pivot (come nel simplesso duale)
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        ratio = []
        for j in range(0, len(self.obj)-1):
            if self.rows[riga][j] >= 0:
                ratio.append(Fraction(-99999999 * abs(max(rhs))))
                continue
            ratio.append(Fraction(self.obj[j], self.rows[riga][j]))
        if max(ratio) < 0:
            return -1
        return argmax(ratio)

    def mostra_tableau(self):
        s = 'xB'
        # Header
        for i in range(len(self.rows[0])-2):
            s += '\tx' + str(int(i+1))
        s += '\tb'
        # Parte interna Tableau
        for i in range(len(self.rows)):
            s += '\nx'+str(int(self.basis[i]))
            for j in range(1,len(self.rows[i])):
                s += '\t' + str(self.rows[i][j])
        # Coefficienti di costo ridotto e funzione obiettivo
        s += '\nc.c.r.'
        for i in range(1,len(self.obj)-1):
            s += '\t' + str(self.obj[i])
        if self.prob_type == 'max':
            s += '\t' + str(self.obj[-1]-self.term_noto_obj)
        elif self.prob_type == 'min':
            s += '\t' + str(self.obj[-1]+self.term_noto_obj)
        print(s)
        self.stampa_soluzione_base_corrente()

    def crea_primo_tableau(self):
        # Compilo il tableau
        for i in range(len(self.rows)):
            self.obj += [0]
            ident = [0 for r in range(len(self.rows))]
            ident[i] = Fraction(1)
            self.rows[i] += ident + [self.cons[i]]
            self.rows[i] = array(self.rows[i])
        self.obj = array(self.obj + [0])

        # Setto le slack come variabili di base
        if self.n >= self.m:
            for i in range(1,self.m+1):
                self.basis += [self.n+i]
        else: # self.n < self.m:
            for i in range(0,self.m):
                self.basis += [self.n+i+1]
            #print(self.basis)

    def _pivot(self, row, col):
        #print('row = ' + str(row))
        #print('col = ' + str(col))
        e = self.rows[row][col]
        print('Elemento pivot a[' + str(row+1) + '][' + str(col) + '] = ' + str(e))
        assert e != 0
        self.rows[row] /= e
        for r in range(len(self.rows)):
            if r == row: continue
            self.rows[r] = self.rows[r] - self.rows[r][col]*self.rows[row]
        #print('prima: ' + str(self.obj))
        self.obj = self.obj - self.obj[col]*self.rows[row]
        #print('dopo: ' + str(self.obj))

    def pivot_colonna_riga(self, c, r):
        self._pivot(r-1,c)
        print('Variabile entrante: x' + str(c))
        print('Variabile uscente: x' + str(self.basis[r-1]))
        self.basis[r-1] = c

    def step_primale(self):
        c = self._colonna_di_max_guadagno()
        r = self._riga_bloccante(c)
        self._pivot(r,c)
        print('Variabile entrante: x' + str(c))
        print('Variabile uscente: x' + str(self.basis[r]))
        self.basis[r] = c

    def step_duale(self):
        print('Iterazione con il simplesso duale')
        r = self._riga_max_inammissibile()
        if r == -1:
            print('Attenzione: nessun termine noto è negativo -> Puoi continuare con il simplesso primale')
            return
        c = self._colonna_bloccante(r)
        if c == -1:
            print('Il duale è illimitato -> Il primale è impossibile -> Fine')
            return
        self._pivot(r,c)
        print('Variabile entrante: x' + str(c))
        print('Variabile uscente: x' + str(self.basis[r]))
        self.basis[r] = c

    def stampa_soluzione_base_corrente(self):
        sol = [0 for x in range(len(self.rows[0])-2)]
        for i in range(len(self.basis)):
            sol[self.basis[i]-1] = self.rows[i][-1]
        s = 'Soluzione di base = ('
        for i in range(len(sol)):
            if i != len(sol)-1:
                s += str(sol[i]) + ', '
            else:
                s += str(sol[i]) + ')'
        print(s)
        if self.prob_type == 'max':
            print('Funzione obiettivo = ' + str(-(self.obj[-1])+self.term_noto_obj))
        elif self.prob_type == 'min':
            print('Funzione obiettivo = ' + str((self.obj[-1])+self.term_noto_obj))


    def prossimo_step(self):
        self.stampa_soluzione_base_corrente()
        if self.is_feasible():
            if self.is_optimal():
                print("Tale soluzione è ammissibile e ottima -> Fine")
            else:
                print("Tale soluzione è ammissibile ma NON è ottima -> Continua con il simplesso primale")
        else:
            if self.is_optimal():
                print("Tale soluzione NON è ammissibile ma soddisfa le condizioni di ottimalità -> Continua con il simplesso duale")
            else:
                print("Tale soluzione NON è né ammissibile né ottima")
